In [33]:
import pandas as pd
import numpy as np

Compile the statistics

In [34]:
df = pd.read_csv('peptide_fit_MixedLM.csv', index_col = 0)

In [35]:
df.head()

,Genes,Oxy8vs21_Log2FC,Oxy8vs21_pval,Oxy8vs21_FDR,Oxy60vs21_Log2FC,Oxy60vs21_pval,Oxy60vs21_FDR
Protein.Group,,,,,,,
A2A8Z1,OSBPL9,-1.295320,0.269415,0.541527,NaN,NaN,NaN
A2AAJ9,OBSCN,-0.166322,0.033156,0.157391,-0.043890,0.889802,0.978119
A2AGT5,CKAP5,0.995426,0.108304,0.332313,NaN,NaN,NaN
A2AHC3,CAMSAP1,-0.232592,0.721692,0.867666,-0.427154,0.267251,0.625109
A2AJI0,MAP7D1,0.271811,0.183617,0.447999,-0.132313,0.554692,0.854480


In [36]:
df.shape

(2108, 7)

In [43]:
df_sub = df[np.logical_and(df.Oxy8vs21_FDR<0.05, df.Oxy8vs21_Log2FC>0)]
df_sub.shape

(186, 7)

In [44]:
df_sub = df[np.logical_and(df.Oxy60vs21_FDR<0.05, df.Oxy60vs21_Log2FC>0)]
df_sub.shape

(30, 7)

In [45]:
df2 = pd.read_csv('peptide_fit.csv', index_col=0)

In [46]:
df2.head()

,Protein.Group,Genes,Oxygen,Kd,Kd_pvalue,Half_life
1,A2A690,TANC2,21,0.067559,1.009979e-03,10.259835
2,A2A8Z1,OSBPL9,8,0.134929,1.451300e-03,5.137122
3,A2A8Z1,OSBPL9,21,0.331158,3.071289e-03,2.093098
4,A2AAJ9,OBSCN,8,0.049183,4.983370e-107,14.093213
5,A2AAJ9,OBSCN,21,0.055193,2.477132e-234,12.558622


In [47]:
print(df2['Protein.Group'].drop_duplicates().shape)
print(df2[df2.Oxygen == 8]['Protein.Group'].drop_duplicates().shape)
print(df2[df2.Oxygen == 21]['Protein.Group'].drop_duplicates().shape)
print(df2[df2.Oxygen == 60]['Protein.Group'].drop_duplicates().shape)

(2805,)
(2413,)
(2249,)
(2232,)


In [48]:
print(df2['Half_life'].median())
print(df2[df2.Oxygen == 8]['Half_life'].median())
print(df2[df2.Oxygen == 21]['Half_life'].median())
print(df2[df2.Oxygen == 60]['Half_life'].median())

7.241997122033391
6.801918686517298
7.488749366774727
7.429891554683203


In [49]:
df2_2 = df2.groupby(['Protein.Group','Genes','Oxygen'])['Kd'].mean().to_frame().reset_index().pivot(index = ['Protein.Group','Genes'],
                                                                                           columns = 'Oxygen',
                                                                                           values = 'Kd')

In [50]:
df2_2 = df2_2.rename(columns = {8:'Kd_mean_8',
                                21: 'Kd_mean_21',
                                60:'Kd_mean_60'})

In [51]:
df2_2.head()

,Oxygen,Kd_mean_8,Kd_mean_21,Kd_mean_60
Protein.Group,Genes,,,
A2A690,TANC2,NaN,0.067559,NaN
A2A8Z1,OSBPL9,0.134929,0.331158,NaN
A2AAJ9,OBSCN,0.049183,0.055193,0.053539
A2AGT5,CKAP5,0.496754,0.249166,NaN
A2AHC3,CAMSAP1,0.282712,0.332171,0.247045


In [52]:
df2_2 = pd.merge(df2_2.reset_index(), df.drop(columns = 'Genes'), how='left', left_on='Protein.Group', right_index=True)

In [53]:
df2_2.head()

,Protein.Group,Genes,Kd_mean_8,Kd_mean_21,Kd_mean_60,Oxy8vs21_Log2FC,Oxy8vs21_pval,Oxy8vs21_FDR,Oxy60vs21_Log2FC,Oxy60vs21_pval,Oxy60vs21_FDR
0,A2A690,TANC2,NaN,0.067559,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A2A8Z1,OSBPL9,0.134929,0.331158,NaN,-1.295320,0.269415,0.541527,NaN,NaN,NaN
2,A2AAJ9,OBSCN,0.049183,0.055193,0.053539,-0.166322,0.033156,0.157391,-0.043890,0.889802,0.978119
3,A2AGT5,CKAP5,0.496754,0.249166,NaN,0.995426,0.108304,0.332313,NaN,NaN,NaN
4,A2AHC3,CAMSAP1,0.282712,0.332171,0.247045,-0.232592,0.721692,0.867666,-0.427154,0.267251,0.625109


In [54]:
df2_2.set_index('Protein.Group').to_csv('Peptide_fit_statistics.csv')

## Coverage

Peptide

In [55]:
df3 = pd.read_csv('peptide_level.csv', index_col =0)

In [56]:
df3 = df3.dropna()

In [57]:
df3['Modified.Sequence'].drop_duplicates().shape

(26652,)

In [58]:
print(df3[df3.Oxygen == 8]['Modified.Sequence'].drop_duplicates().shape)
print(df3[df3.Oxygen == 21]['Modified.Sequence'].drop_duplicates().shape)
print(df3[df3.Oxygen == 60]['Modified.Sequence'].drop_duplicates().shape)

(26393,)
(26563,)
(26390,)


Peptide and protein (filtered)

In [59]:
df4 = pd.read_csv('peptide_level_filtered.csv', index_col = 0)

In [60]:
df4['Modified.Sequence'].drop_duplicates().shape

(14737,)

In [61]:
print(df4[df4.Oxygen == 8]['Modified.Sequence'].drop_duplicates().shape)
print(df4[df4.Oxygen == 21]['Modified.Sequence'].drop_duplicates().shape)
print(df4[df4.Oxygen == 60]['Modified.Sequence'].drop_duplicates().shape)

(10806,)
(8945,)
(9316,)


In [62]:
print(df4['Protein.Group'].drop_duplicates().shape)
print(df4[df4.Oxygen == 8]['Protein.Group'].drop_duplicates().shape)
print(df4[df4.Oxygen == 21]['Protein.Group'].drop_duplicates().shape)
print(df4[df4.Oxygen == 60]['Protein.Group'].drop_duplicates().shape)

(2849,)
(2466,)
(2285,)
(2293,)
